In [1]:
import cv2
import matplotlib.pyplot as plt
import os
import glob
import numpy as np
import tensorflow as tf
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import zipfile
import shutil

# ImageDataGenerator

In [2]:
import cv2
from google.colab.patches import cv2_imshow

In [3]:
# 데이터 증강
# 직접 가지고 가서 학습할 수 있도록 구현!!!!
train_datagen = ImageDataGenerator(
    rescale = 1./255. ,
    validation_split = 0.2,
    zoom_range = 0.2, # zoom_range 확대 정도, shear_range 기울기
)
valid_datagen = ImageDataGenerator(
    rescale = 1./255. ,
    validation_split = 0.2,
)

In [4]:
# input : (224,224,3) output(1,1,2)
image_height = 224
image_width = 224

# train set : image_collection, train answer set : train_data_sub_folder
train_data_sub_folder = "/content/drive/MyDrive/2023_T_Academy/07_deep_learning/project/data/image_classification"
train_generator = train_datagen.flow_from_directory(
    directory = train_data_sub_folder,
    subset = "training",
    batch_size = 32,
    seed = 42, shuffle = True,
    class_mode = "categorical",
    target_size = (image_height, image_width)
)

valid_generator = valid_datagen.flow_from_directory(
    directory = train_data_sub_folder,
    subset = "validation",
    batch_size = 32,
    seed = 42, shuffle = True,
    class_mode = "categorical",
    target_size = (image_height, image_width)
)

Found 754 images belonging to 2 classes.
Found 2938 images belonging to 2 classes.


In [5]:
# 논문 보고 구조 가져오기
# input : (224,224,3) output(1,1,1000) -> input : (224,224,3) output(1,1,2)
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D( input_shape = (224,224,3),kernel_size=(3,3), filters=6 ),

    tf.keras.layers.Conv2D(
        kernel_size = (5,5),
        filters = 64,
        strides = (2,2), # output 인 channel 사이즈를 반토막
        padding = "valid",
        activation = "relu", # 최근 추가
    ),
    tf.keras.layers.MaxPooling2D(pool_size = (2,2), strides=(2,2)),

    tf.keras.layers.Conv2D(
        kernel_size = (5,5),
        filters = 128,
        strides = (2,2), # output 인 channel 사이즈를 반토막
        padding = "valid",
        activation = "relu", # 최근 추가
    ),
    tf.keras.layers.MaxPooling2D(pool_size = (2,2), strides=(2,2)),

    tf.keras.layers.Conv2D(
        kernel_size = (5,5),
        filters = 256,
        strides = (2,2), # output 인 channel 사이즈를 반토막
        padding = "valid",
        activation = "relu", # 최근 추가
    ),
    tf.keras.layers.MaxPooling2D(pool_size = (2,2), strides=(2,2)),
    tf.keras.layers.Flatten(), # 빼먹엇

    tf.keras.layers.Dropout(rate=0.3),# dropout : 입력 뉴런의 30%를 학습 중에 무작위로 비활성화
    tf.keras.layers.Dense( units = 2, activation="softmax")
])

In [6]:
# batch로 넣기 , metrics=["precison"]
from keras.engine.training import optimizer
model.compile( optimizer="Adam",
              loss = "categorical_crossentropy",
              metrics=[tf.keras.metrics.Precision()]) # accuracy : 0~1

history_generator = model.fit(train_generator,
                    epochs=50)

Epoch 1/50
24/24 [==============================] - 231s 9s/step - loss: 0.0256 - precision: 0.9894
Epoch 2/50
24/24 [==============================] - 83s 3s/step - loss: 0.0000e+00 - precision: 1.0000
Epoch 3/50
24/24 [==============================] - 77s 3s/step - loss: 0.0000e+00 - precision: 1.0000
Epoch 4/50
24/24 [==============================] - 77s 3s/step - loss: 0.0000e+00 - precision: 1.0000
Epoch 5/50
24/24 [==============================] - 77s 3s/step - loss: 0.0000e+00 - precision: 1.0000
Epoch 6/50
24/24 [==============================] - 77s 3s/step - loss: 0.0000e+00 - precision: 1.0000
Epoch 7/50
24/24 [==============================] - 79s 3s/step - loss: 0.0000e+00 - precision: 1.0000
Epoch 8/50
24/24 [==============================] - 78s 3s/step - loss: 0.0000e+00 - precision: 1.0000
Epoch 9/50
24/24 [==============================] - 82s 3s/step - loss: 0.0000e+00 - precision: 1.0000
Epoch 10/50
24/24 [==============================] - 77s 3s/step - loss: 0.0

In [ ]:
# batch로 넣기, metrics=["accuracy"]
from keras.engine.training import optimizer
model.compile( optimizer="Adam",
              loss = "categorical_crossentropy",
              metrics=[tf.keras.metrics.Precision()]) # accuracy : 0~1

history_generator = model.fit(train_generator,
                    epochs=50)

In [ ]:
# 라벨링 안한 거 넣어서 확인해보기